# Glassdoor - Companies and Ratings

In [1]:
import  requests
import urllib.request
import csv
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import time

In [2]:
#Employer.csv contains all the employers retrieved via web scraping job posts. 
#Here will be using those values for the 'employer' parameter while making request via Glassdoor API

employer_df = pd.read_csv("Employer.csv")
print(employer_df.shape[0])

1579


In [3]:
employer_df[327:328]

,Employer
327,Conrad Consulting


### Getting ratings for the companies in the Employer.csv

We are using public Glassdoor API id (t.p=25738) and key (t.k=iRCtcWJQamE) provided via blog post: [Get Glassdoor Data from their API using Python](https://oonlab.com/get-glassdoor-data-from-their-api-using-python). This API key was gained 3 years ago when Glassdoor had free API sign up which they don’t provide any more until further notice [(link)](https://help.glassdoor.com/article/Glassdoor-API/en_US). Also, this API key has come with following restrictions on Glassdoor API Usage: [Public API Terms of Use](https://www.glassdoor.com/crs/api/glassdoor-public-api-terms.pdf). We have decided to use this public API id and key until we get our partnership inquiry permitted.

Note: We have submitted [partnership inquiry](https://help.glassdoor.com/ContactUs/en_US) several times during the fits month of Final Team Software Project, but with no respond from Glassdoor.

Glassdoor API request, used in the blog post, retrieves data about companies ratings. By reading [Glassdoor API Overview](https://www.glassdoor.com/developer/index.htm) and guidance in regards to parameters, we come to conclusion that by changing value of parameter ‘action’ from ‘action = employer’ to:
‘action = salaries’ we will fetch data about salaries, and
‘action = interviews’ we will fetch data about interview difficulty.

#### Job Postings Data
We weren’t able to find a value of parameter ‘action’ that would retrieve data about job postings, hence we decided to use web scraping for that quest. Tech topics from Meetup were used to retrieve corresponding job posts for Dublin. 

In [4]:
URLs=[]
for i in range(employer_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'employers','employer':employer_df['Employer'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    #print(r.url)
    URLs.append(r.url)
    
len(URLs)

1579

In [5]:
URLs[327]

'http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Conrad+Consulting'

In [6]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ratings_list_nested = []
for i in range(len(URLs)):
    url = URLs[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    ratings_data=json.loads(soup.p.get_text())
    ratings_list_nested.append(pd.DataFrame(ratings_data['response']['employers']))

In [19]:
ratings_list_nested[400]

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,id,industry,industryId,industryName,isEEP,...,numberOfRatings,overallRating,ratingDescription,recommendToFriendRating,sectorId,sectorName,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,2.9,"{'name': 'Ian Filby', 'title': 'CEO', 'numberO...",3.1,2.7,True,12458,Home Furniture & Housewares Stores,200112,Home Furniture & Housewares Stores,False,...,40,3.3,OK,46,10022,Retail,2.5,https://media.glassdoor.com/sqll/12458/dfs-fur...,www.dfs.co.uk,2.3


In [8]:
len(ratings_list_nested)

1579

In [9]:
company_rating_df_nested = pd.DataFrame()
for i in range(len(ratings_list_nested)):
    company_rating_df_nested=company_rating_df_nested.append(ratings_list_nested[i])
company_rating_df_nested

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,id,industry,industryId,industryName,...,overallRating,parentEmployer,ratingDescription,recommendToFriendRating,sectorId,sectorName,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,2.7,"{'name': 'Ranjit Singh', 'title': 'CEO', 'numb...",2.2,2.5,False,{'attributionURL': 'https://www.glassdoor.com/...,523766,Food & Beverage Manufacturing,200071.0,Food & Beverage Manufacturing,...,3.0,NaN,OK,49,10015.0,Manufacturing,2.4,https://media.glassdoor.com/sqll/523766/2-sist...,www.2sfg.com,2.6
0,5.0,NaN,4.4,5.0,True,NaN,1839855,IT Services,200064.0,IT Services,...,4.8,NaN,Very Satisfied,100,10013.0,Information Technology,5.0,https://media.glassdoor.com/sqll/1839855/2sour...,www.2sourcetalent.com,4.4
0,3.0,"{'name': 'Robert Finnegan', 'title': 'CEO', 'n...",3.4,3.1,False,{'attributionURL': 'https://www.glassdoor.com/...,39331,Telecommunications Services,200122.0,Telecommunications Services,...,3.2,NaN,OK,57,10023.0,Telecommunications,2.7,https://media.glassdoor.com/sqll/39331/3-irela...,www.three.ie,3.3
0,2.1,"{'name': 'Damian Doherty', 'title': 'Owner', '...",2.3,2.3,True,{'attributionURL': 'https://www.glassdoor.com/...,1193024,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,2.8,NaN,OK,49,10006.0,Business Services,2.8,https://media.glassdoor.com/sqll/1193024/3d-pe...,www.3dpersonnel.com,2.5
0,5.0,NaN,5.0,5.0,False,NaN,1169343,,NaN,NaN,...,5.0,NaN,Very Satisfied,100,NaN,NaN,5.0,,www.aattech.com,5.0
0,3.3,"{'name': 'Ulrich Spiesshofer', 'title': 'Chief...",3.3,3.6,True,{'attributionURL': 'https://www.glassdoor.com/...,3076,Electrical & Electronic Manufacturing,200070.0,Electrical & Electronic Manufacturing,...,3.6,NaN,Satisfied,72,10015.0,Manufacturing,3.0,https://media.glassdoor.com/sqll/3076/abb-squa...,www.abb.com,3.6
1,3.0,"{'name': 'Angel Alvarez', 'title': 'Chief Exec...",3.3,3.0,False,NaN,485895,Wholesale,200033.0,Wholesale,...,3.1,NaN,OK,51,10006.0,Business Services,2.9,https://media.glassdoor.com/sqll/485895/abb-co...,www.abboptical.com,3.2
2,3.3,NaN,3.9,3.9,False,NaN,748075,Consulting,200028.0,Consulting,...,3.9,NaN,Satisfied,100,10006.0,Business Services,3.5,,www.abb-consultant.com,3.6
3,5.0,NaN,5.0,5.0,False,NaN,908157,Health Care Services & Hospitals,200059.0,Health Care Services & Hospitals,...,5.0,NaN,Very Satisfied,100,10012.0,Health Care,5.0,,www.tropos.com,5.0
4,2.3,"{'name': 'Matthias Heilmann', 'title': 'Produc...",3.1,2.4,False,NaN,3233,Computer Hardware & Software,200060.0,Computer Hardware & Software,...,2.3,"{'id': 3076, 'name': 'ABB', 'relationshipDate'...",Dissatisfied,29,10013.0,Information Technology,2.0,https://media.glassdoor.com/sqll/3233/ventyx-s...,www.abb.com/enterprise-software,2.8


In [11]:
#Droping columns 'isEEP' and 'industry'
company_rating_df_nested=company_rating_df_nested.drop(['isEEP','industry'], axis=1)

In [44]:
#Renaming colums
company_rating_df_nested=company_rating_df_nested.rename(index=str, columns={"name": "company_name", "id": "company_id", "squareLogo": "company_logo_url"})

In [13]:
#Adding new columns because of the nested featuredReview column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list=['review_URL','review_ID','review_currentJob','review_DateTime',
              'review_jobTitle','review_location','review_pros','review_cons','review_overall']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list])

In [14]:
split_featuredReview = company_rating_df_nested['featuredReview'].apply(pd.Series) # split the featuredReview column from company_rating_df_nested
split_featuredReview = split_featuredReview.reset_index(drop=True) # reset index

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['review_URL'].iloc[i]=split_featuredReview['attributionURL'][i]
    company_rating_df_nested['review_ID'].iloc[i]=split_featuredReview['id'][i]
    company_rating_df_nested['review_currentJob'].iloc[i]=split_featuredReview['currentJob'][i]
    company_rating_df_nested['review_DateTime'].iloc[i]=split_featuredReview['reviewDateTime'][i]
    company_rating_df_nested['review_jobTitle'].iloc[i]=split_featuredReview['jobTitle'][i]
    company_rating_df_nested['review_location'].iloc[i]=split_featuredReview['location'][i]
    company_rating_df_nested['review_pros'].iloc[i]=split_featuredReview['pros'][i]
    company_rating_df_nested['review_cons'].iloc[i]=split_featuredReview['cons'][i]
    company_rating_df_nested['review_overall'].iloc[i]=split_featuredReview['overall'][i]


/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
#Adding new columns because of the nested ceo column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list2=['ceo_name','ceo_title','ceo_numberOfRatings','ceo_approval_rate','ceo_Photo_URL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list2])

In [16]:
split_ceo = company_rating_df_nested['ceo'].apply(pd.Series) # split the ceo column from company_rating_df_nested
split_ceo = split_ceo.reset_index(drop=True) # reset index
split_ceo_2 = split_ceo['image'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['ceo_name'].iloc[i]=split_ceo['name'][i]
    company_rating_df_nested['ceo_title'].iloc[i]=split_ceo['title'][i]
    company_rating_df_nested['ceo_numberOfRatings'].iloc[i]=split_ceo['numberOfRatings'][i]
    company_rating_df_nested['ceo_approval_rate'].iloc[i]=split_ceo['pctApprove'][i]
    company_rating_df_nested['ceo_Photo_URL'].iloc[i]=split_ceo_2['src'][i]
#company_rating_df_nested.head()

/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
#Adding new columns because of the nested parentEmployer column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list3=['parentEmployer_name','parentEmployer_ID','parentEmployer_logoURL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list3])

In [36]:
split_parentEmployer = company_rating_df_nested['parentEmployer'].apply(pd.Series) # split the parentEmployer column from company_rating_df_nested
split_parentEmployer = split_parentEmployer.reset_index(drop=True) # reset index
split_parentEmployer_2 = split_parentEmployer['logo'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['parentEmployer_name'].iloc[i]=split_parentEmployer['name'][i]
    company_rating_df_nested['parentEmployer_ID'].iloc[i]=split_parentEmployer['id'][i]
    company_rating_df_nested['parentEmployer_logoURL'].iloc[i]=split_parentEmployer_2['normalUrl'][i]


#company_rating_df_nested.head()

/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
company_rating_df_nested

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,company_id,industryId,industryName,company_name,...,review_cons,review_overall,ceo_name,ceo_title,ceo_numberOfRatings,ceo_approval_rate,ceo_Photo_URL,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL
0,2.7,"{'name': 'Ranjit Singh', 'title': 'CEO', 'numb...",2.2,2.5,False,{'attributionURL': 'https://www.glassdoor.com/...,523766,200071.0,Food & Beverage Manufacturing,2 Sisters Food Group,...,"Night shifts, noisy and due to quality require...",4.0,Ranjit Singh,CEO,24.0,65.0,https://media.glassdoor.com/people/sqll/523766...,NaN,NaN,NaN
0,5.0,NaN,4.4,5.0,True,NaN,1839855,200064.0,IT Services,2Source Talent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3.0,"{'name': 'Robert Finnegan', 'title': 'CEO', 'n...",3.4,3.1,False,{'attributionURL': 'https://www.glassdoor.com/...,39331,200122.0,Telecommunications Services,Three Ireland,...,"Car park is usually packed, if you are first i...",5.0,Robert Finnegan,CEO,38.0,63.0,https://media.glassdoor.com/people/sqll/39331/...,NaN,NaN,NaN
0,2.1,"{'name': 'Damian Doherty', 'title': 'Owner', '...",2.3,2.3,True,{'attributionURL': 'https://www.glassdoor.com/...,1193024,200032.0,Staffing & Outsourcing,3D Personnel,...,There can be days you are available for work a...,3.0,Damian Doherty,Owner,3.0,68.0,NaN,NaN,NaN,NaN
0,5.0,NaN,5.0,5.0,False,NaN,1169343,NaN,NaN,AAT Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3.3,"{'name': 'Ulrich Spiesshofer', 'title': 'Chief...",3.3,3.6,True,{'attributionURL': 'https://www.glassdoor.com/...,3076,200070.0,Electrical & Electronic Manufacturing,ABB,...,Low salary and slow paced,4.0,Ulrich Spiesshofer,Chief Executive Officer,731.0,83.0,https://media.glassdoor.com/people/sqll/3076/a...,NaN,NaN,NaN
1,3.0,"{'name': 'Angel Alvarez', 'title': 'Chief Exec...",3.3,3.0,False,NaN,485895,200033.0,Wholesale,ABB Optical Group,...,NaN,NaN,Angel Alvarez,Chief Executive Officer,51.0,77.0,NaN,NaN,NaN,NaN
2,3.3,NaN,3.9,3.9,False,NaN,748075,200028.0,Consulting,ABB Consultant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,5.0,5.0,False,NaN,908157,200059.0,Health Care Services & Hospitals,ABB Tropos Wireless Communication Systems,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.3,"{'name': 'Matthias Heilmann', 'title': 'Produc...",3.1,2.4,False,NaN,3233,200060.0,Computer Hardware & Software,Ventyx,...,NaN,NaN,Matthias Heilmann,Product Group Manager,16.0,12.0,NaN,ABB,3076.0,https://media.glassdoor.com/sql/3076/abb-squar...


In [51]:
#dropping columns'ceo','featuredReview', 'parentEmployer'
company_rating_df2=company_rating_df_nested.drop(['ceo','featuredReview', 'parentEmployer'], axis=1)

In [52]:
#Removing duplicate rows
company_rating_df_no_duplicates = company_rating_df2.drop_duplicates(subset=None, keep='first', inplace=False)

In [54]:
#Changing order of the columns
company_rating_df = company_rating_df_no_duplicates[['company_name', 'company_id', 'website', 'company_logo_url', 
                                              'parentEmployer_name', 'parentEmployer_ID','parentEmployer_logoURL',
                                              'numberOfRatings', 'overallRating', 'ratingDescription',
                                              'workLifeBalanceRating', 'careerOpportunitiesRating', 
                                              'compensationAndBenefitsRating', 'cultureAndValuesRating',
                                              'recommendToFriendRating', 'seniorLeadershipRating', 'ceo_name', 
                                              'ceo_title', 'ceo_Photo_URL', 'ceo_numberOfRatings', 'ceo_approval_rate',
                                              'review_URL', 'review_ID', 'review_pros', 'review_cons', 'review_overall',
                                              'review_location', 'review_DateTime', 'review_currentJob', 
                                              'review_jobTitle', 'review_location', 'industryName', 'industryId',
                                              'sectorName', 'sectorId', 'exactMatch']]

In [56]:
company_rating_df.to_csv('company_overall_ratings_all.csv', sep = ',', index = False) # create csv for salaries
company_rating_df

,company_name,company_id,website,company_logo_url,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL,numberOfRatings,overallRating,ratingDescription,...,review_location,review_DateTime,review_currentJob,review_jobTitle,review_location,industryName,industryId,sectorName,sectorId,exactMatch
0,2 Sisters Food Group,523766,www.2sfg.com,https://media.glassdoor.com/sqll/523766/2-sist...,NaN,NaN,NaN,54,3.0,OK,...,"Longford, Co. Longford (Ireland)",2015-11-24 00:29:59.22,False,Employee,"Longford, Co. Longford (Ireland)",Food & Beverage Manufacturing,200071.0,Manufacturing,10015.0,False
0,2Source Talent,1839855,www.2sourcetalent.com,https://media.glassdoor.com/sqll/1839855/2sour...,NaN,NaN,NaN,5,4.8,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,IT Services,200064.0,Information Technology,10013.0,True
0,Three Ireland,39331,www.three.ie,https://media.glassdoor.com/sqll/39331/3-irela...,NaN,NaN,NaN,65,3.2,OK,...,"Dublin, Co. Dublin (Ireland)",2018-05-22 01:07:23.073,True,Employee,"Dublin, Co. Dublin (Ireland)",Telecommunications Services,200122.0,Telecommunications,10023.0,False
0,3D Personnel,1193024,www.3dpersonnel.com,https://media.glassdoor.com/sqll/1193024/3d-pe...,NaN,NaN,NaN,6,2.8,OK,...,"Dublin, Co. Dublin (Ireland)",2017-11-14 02:21:03.443,True,General Operator,"Dublin, Co. Dublin (Ireland)",Staffing & Outsourcing,200032.0,Business Services,10006.0,True
0,AAT Technology,1169343,www.aattech.com,,NaN,NaN,NaN,1,5.0,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
0,ABB,3076,www.abb.com,https://media.glassdoor.com/sqll/3076/abb-squa...,NaN,NaN,NaN,2426,3.6,Satisfied,...,,2017-06-30 08:22:17.303,False,Employee,,Electrical & Electronic Manufacturing,200070.0,Manufacturing,10015.0,True
1,ABB Optical Group,485895,www.abboptical.com,https://media.glassdoor.com/sqll/485895/abb-co...,NaN,NaN,NaN,85,3.1,OK,...,NaN,NaN,NaN,NaN,NaN,Wholesale,200033.0,Business Services,10006.0,False
2,ABB Consultant,748075,www.abb-consultant.com,,NaN,NaN,NaN,4,3.9,Satisfied,...,NaN,NaN,NaN,NaN,NaN,Consulting,200028.0,Business Services,10006.0,False
3,ABB Tropos Wireless Communication Systems,908157,www.tropos.com,,NaN,NaN,NaN,1,5.0,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,Health Care Services & Hospitals,200059.0,Health Care,10012.0,False
4,Ventyx,3233,www.abb.com/enterprise-software,https://media.glassdoor.com/sqll/3233/ventyx-s...,ABB,3076.0,https://media.glassdoor.com/sql/3076/abb-squar...,194,2.3,Dissatisfied,...,NaN,NaN,NaN,NaN,NaN,Computer Hardware & Software,200060.0,Information Technology,10013.0,False
